### Numerical solutions

### 1. GARCH vol with no trading costs
Verify that numerical solution is identical to
$v_t=\frac{\Omega_{t+1}^{-1}\textbf{1}}{\textbf{1}'\Omega_{t+1}^{-1} \textbf{1}}$

Solve:
$V_t(v_t)=\min_{v_t}\Bigg[(1-\rho)\bigg({\frac{1}{2}v_{t}'\Omega_{t+1}  v_{t}}+\mathbb{E}_t[V(v_{t+1})]\bigg) \Bigg] - \kappa_1(v_t'\textbf{1}-1)$

In [125]:
import quadprog
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import numpy as np
import pandas as pd
from numpy.linalg import multi_dot as mdot
from numpy import dot
from numpy import divide
from numpy.linalg import inv

In [128]:
# Setup
rho = 0.0000
Omega_t_plus_1 = pd.read_csv("Omega_t.csv", sep=";", header=None).values

In [129]:
def eq_constraint(v_t):
    return np.sum(v_t)-1

In [130]:
def V_t(v_t, Omega_t_plus_1, rho):
    res = (1-rho)*(0.5*mdot([v_t.T, Omega_t_plus_1, v_t]))
    return res

p = Omega_t_plus_1.shape[0]
constraint = [{'type': 'eq', 'fun': eq_constraint}]
res =minimize(fun = V_t, 
         x0 = np.full((p, 1), 1/p),
         args = (Omega_t_plus_1, rho),
         constraints=constraint,
         method="SLSQP")
v_t_num = res.x

### Comparing to analytical

In [134]:
v_t_num

array([ 0.12930634,  1.01402932, -0.16818878,  0.02485312])

In [131]:
ones = np.ones((p, 1))
v_t_anal = divide(dot(inv(Omega_t_plus_1), ones), mdot([ones.T, inv(Omega_t_plus_1), ones]))

In [132]:
mdot([v_t_anal.T, Omega_t_plus_1, v_t_anal])

array([[0.07023609]])

In [133]:
mdot([v_t_num.T, Omega_t_plus_1, v_t_num])

0.0702360936446367

### Testing the static case without GARCH

In [121]:
def eq_constraint(v_t):
    return np.sum(v_t)-1

def V_t(v_t, Sigma):
    res = 0.5*mdot([v_t.T, Sigma, v_t])
    return res

p = Omega_t_plus_1.shape[0]
constraint = [{'type': 'eq', 'fun': eq_constraint}]
res =minimize(fun = V_t, 
         x0 = np.full((p, 1), 1/p),
         args = (Omega_t_plus_1),
         constraints=constraint,
         method="SLSQP")
v_t_num = res.x

In [122]:
v_t_num

array([ 0.12930634,  1.01402932, -0.16818878,  0.02485312])

In [126]:
v_t_anal = divide(dot(inv(Omega_t_plus_1), ones), mdot([ones.T, inv(Omega_t_plus_1), ones]))

In [127]:
v_t_anal

array([[ 0.12930769],
       [ 1.01402698],
       [-0.1681879 ],
       [ 0.02485323]])